# Geoparsing Demo: Extracting Locations from Text

## Introduction

In the spatial search notebook, we worked with documents that already had geographic coordinates (the Geograph photos were geotagged by photographers). But what if we didn't have these coordinates? What if we only had the text descriptions?

This notebook demonstrates **geoparsing** - the process of automatically:
1. **Recognizing** place names in unstructured text
2. **Resolving** them to geographic coordinates

We'll use the [Irchel Geoparser](https://docs.geoparser.app/) library to extract locations from Geograph photo descriptions, then evaluate how accurately these extracted locations match the tagged photo coordinates.


## Setup

First, let's import the required libraries and load our data.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Geoparser imports
from geoparser import Geoparser
from geoparser.modules import SpacyRecognizer, SentenceTransformerResolver

# For coordinate calculations
from math import radians, cos, sin, asin, sqrt

print("Libraries imported successfully!")


## Load the Geograph Corpus

We'll load the same dataset used in the spatial search notebook - geotagged photographs with text descriptions.


In [ ]:
data_folder = Path('./data')
fn1 = data_folder / 'geograph_mini_corpus.csv'

geograph = pd.read_csv(fn1, encoding='latin-1')

# For this demo, we'll work with the longest texts (better context for geoparsing)
# The Irchel Geoparser uses context to resolve toponyms, so longer texts generally work better
sample_size = 100

# Calculate text length and get the n longest texts
geograph['text_length'] = geograph['text'].str.len()
geograph_sorted = geograph.sort_values('text_length', ascending=False)

# Select the n longest texts, then restore original order
sample = geograph_sorted.head(sample_size).sort_index().reset_index(drop=True)
sample = sample.drop(columns=['text_length'])  # Remove helper column

print(f'Loaded {len(sample)} documents for geoparsing (longest texts for better context).')
print(f'\nExample document:')
print(f"\nID: {sample.iloc[0]['id']}")
print(f"\nTitle: {sample.iloc[0]['title']}")
print(f"\nText: {sample.iloc[0]['text']}")
print(f"\nPhoto Location: ({sample.iloc[0]['lat']:.4f}, {sample.iloc[0]['lon']:.4f})")


## Initialize the Geoparser

We'll configure the Irchel Geoparser with:
- [SpacyRecognizer](https://docs.geoparser.app/en/latest/guides/modules.html#spacyrecognizer)
- [SentenceTransformerResolver](https://docs.geoparser.app/en/latest/guides/modules.html#sentencetransformerresolver)


In [ ]:
print("Initializing geoparser components...")

# Initialize recognizer (identifies place names in text)
recognizer = SpacyRecognizer()

# Initialize resolver (links place names to coordinates)
resolver = SentenceTransformerResolver(min_similarity=0.5)

# Create geoparser instance
geoparser = Geoparser(recognizer=recognizer, resolver=resolver)

print("Geoparser ready!")


## Process Documents with Geoparser

Now we'll process each document to extract place names and their coordinates. We'll combine the title and text for better context.


In [ ]:
# Prepare texts for geoparsing (combine title and text for better context)
texts = []
for idx, row in sample.iterrows():
    title = str(row['title']) if pd.notna(row['title']) else ""
    text = str(row['text']) if pd.notna(row['text']) else ""
    combined = f"{title}. {text}".strip()
    texts.append(combined)

print(f"Processing {len(texts)} documents...")
print("This may take a few minutes...\n")

# Process all documents
documents = geoparser.parse(texts)

print(f"\nProcessed {len(documents)} documents!")


## Extract Results

Let's examine what the geoparser found. We'll extract all resolved toponyms for each document and calculate their distance from the true photo location.


In [ ]:
# Define haversine distance function first (we'll need it)
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees).
    Returns distance in kilometers.
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers
    r = 6371
    
    return c * r

# Store results with ALL toponyms for each document
results = []

for idx, doc in enumerate(documents):
    doc_id = sample.iloc[idx]['id']
    true_lat = sample.iloc[idx]['lat']
    true_lon = sample.iloc[idx]['lon']
    
    # Get all resolved toponyms
    toponyms = [t for t in doc.toponyms if t.location is not None]
    
    # Store each toponym with its details
    toponyms_data = []
    for toponym in toponyms:
        location = toponym.location
        topo_lat = location.data.get('latitude')
        topo_lon = location.data.get('longitude')
        
        # Calculate distance from true location
        distance = haversine_distance(true_lat, true_lon, topo_lat, topo_lon)
        
        toponyms_data.append({
            'text': toponym.text,
            'name': location.data.get('name'),
            'lat': topo_lat,
            'lon': topo_lon,
            'country': location.data.get('country_name'),
            'feature_class': location.data.get('feature_class'),
            'feature_code': location.data.get('feature_code'),
            'admin1': location.data.get('admin1_name'),
            'admin2': location.data.get('admin2_name'),
            'population': location.data.get('population', 0),
            'distance_km': distance
        })
    
    result = {
        'doc_id': doc_id,
        'true_lat': true_lat,
        'true_lon': true_lon,
        'text': texts[idx],
        'num_toponyms': len(toponyms),
        'toponyms': toponyms_data  # Store all toponym data
    }
    
    results.append(result)

results_df = pd.DataFrame(results)

docs_with_toponyms = len(results_df[results_df['num_toponyms'] > 0])
print(f"Toponyms successfully resolved in {docs_with_toponyms} out of {len(results_df)} documents")
print(f"Average toponyms per document: {results_df['num_toponyms'].mean():.2f}")


## Example Results

Let's look at some examples to understand what the geoparser extracted.


In [ ]:
# Show some examples with ALL resolved toponyms
examples = results_df[results_df['num_toponyms'] > 0].sample(5)

for idx, row in examples.iterrows():
    print(f"\n{'='*80}")
    print(f"Document {row['doc_id']}:")
    print(f"Text: {row['text']}")
    print(f"\nPhoto location: ({row['true_lat']:.4f}, {row['true_lon']:.4f})")
    print(f"\nExtracted toponyms ({len(row['toponyms'])}):")
    
    for i, topo in enumerate(row['toponyms'], 1):
        # Build location string
        location_parts = [topo['name']]
        if topo['admin2']:
            location_parts.append(topo['admin2'])
        if topo['admin1']:
            location_parts.append(topo['admin1'])
        if topo['country']:
            location_parts.append(topo['country'])
        
        feature_info = f" [{topo['feature_code']}]" if topo['feature_code'] else ""
        
        print(f"\n  {i}. Toponym text: '{topo['text']}'")
        print(f"     Resolved entity: {', '.join(location_parts)}{feature_info}")
        print(f"     Resolved coordinates: ({topo['lat']:.4f}, {topo['lon']:.4f})")
        print(f"     Distance from photo location: {topo['distance_km']:.2f} km")


## Evaluation: Comparing Different Resolution Strategies

Now we need to decide how to determine the predicted location for each document based on the extracted toponyms. We'll compare three different strategies:

1. **First Mention**: Simply use the first toponym's location
2. **Centroid**: Calculate the average (centroid) of all toponym locations
3. **Domain-Specific Rule**: Use domain knowledge that our texts are about UK locations - select the first toponym that is in the UK

Let's implement all three and compare their performance.


In [ ]:
# Strategy 1: First Mention
strategy1_errors = []
for _, row in results_df[results_df['num_toponyms'] > 0].iterrows():
    first_topo = row['toponyms'][0]
    strategy1_errors.append(first_topo['distance_km'])

# Strategy 2: Centroid
strategy2_errors = []
for _, row in results_df[results_df['num_toponyms'] > 0].iterrows():
    if len(row['toponyms']) == 1:
        # If only one toponym, same as first
        strategy2_errors.append(row['toponyms'][0]['distance_km'])
    else:
        # Calculate centroid
        avg_lat = np.mean([t['lat'] for t in row['toponyms']])
        avg_lon = np.mean([t['lon'] for t in row['toponyms']])
        error = haversine_distance(row['true_lat'], row['true_lon'], avg_lat, avg_lon)
        strategy2_errors.append(error)

# Strategy 3: Domain-Specific Rule
# Use domain knowledge that texts are about UK locations
strategy3_errors = []
for _, row in results_df[results_df['num_toponyms'] > 0].iterrows():
    # Try to find the first UK toponym
    uk_toponym = None
    for topo in row['toponyms']:
        if topo['country'] == 'United Kingdom':
            uk_toponym = topo
            break
    
    # If we found a UK toponym, use it; otherwise fall back to first toponym
    if uk_toponym is not None:
        strategy3_errors.append(uk_toponym['distance_km'])
    else:
        # No UK toponym found, use first one
        strategy3_errors.append(row['toponyms'][0]['distance_km'])

print(f"Calculated errors for {len(strategy1_errors)} documents using 3 strategies")


## Comparing Strategy Performance

Let's compute evaluation metrics for each of the three strategies.


In [ ]:
def calculate_metrics(errors, strategy_name):
    """Calculate and return evaluation metrics for a strategy"""
    errors_array = np.array(errors)
    
    metrics = {
        'strategy': strategy_name,
        'mean': np.mean(errors_array),
        'median': np.median(errors_array),
        'std': np.std(errors_array),
        'min': np.min(errors_array),
        'max': np.max(errors_array),
        'acc_1km': (errors_array <= 1).sum() / len(errors_array) * 100,
        'acc_10km': (errors_array <= 10).sum() / len(errors_array) * 100,
        'acc_161km': (errors_array <= 161).sum() / len(errors_array) * 100
    }
    return metrics

# Calculate metrics for all strategies
metrics_s1 = calculate_metrics(strategy1_errors, "Strategy 1: First Mention")
metrics_s2 = calculate_metrics(strategy2_errors, "Strategy 2: Centroid")
metrics_s3 = calculate_metrics(strategy3_errors, "Strategy 3: Domain-Specific Rule")

# Print comparison
total_docs = len(results_df)
docs_with_toponyms = len(strategy1_errors)

print("\n" + "="*80)
print("GEOPARSING EVALUATION: COMPARING RESOLUTION STRATEGIES")
print("="*80)
print(f"\nResolution Rate: {docs_with_toponyms}/{total_docs} ({docs_with_toponyms/total_docs*100:.1f}%)")
print(f"Documents with at least one resolved toponym\n")

for metrics in [metrics_s1, metrics_s2, metrics_s3]:
    print("-" * 80)
    print(f"\n{metrics['strategy']}")
    print(f"\nDistance Error Statistics (km):")
    print(f"  Mean:   {metrics['mean']:7.2f} km")
    print(f"  Median: {metrics['median']:7.2f} km")
    print(f"  Std:    {metrics['std']:7.2f} km")
    print(f"  Min:    {metrics['min']:7.2f} km")
    print(f"  Max:    {metrics['max']:7.2f} km")
    print(f"\nAccuracy at Distance Thresholds:")
    print(f"  Within 1 km:    {metrics['acc_1km']:5.1f}%")
    print(f"  Within 10 km:   {metrics['acc_10km']:5.1f}%")
    print(f"  Within 161 km:  {metrics['acc_161km']:5.1f}%")
    print()

print("="*80)

# Store metrics for plotting
all_metrics = [metrics_s1, metrics_s2, metrics_s3]
all_errors = [strategy1_errors, strategy2_errors, strategy3_errors]


## Visualize Error Distributions

Let's visualize the error distributions for all three strategies side-by-side.


In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 14))

strategy_names = ['First Mention', 'Centroid', 'Domain-Specific Rule']
colors = ['steelblue', 'coral', 'mediumseagreen']

# Calculate common x and y axis limits for all plots
all_errors_combined = strategy1_errors + strategy2_errors + strategy3_errors
x_max = max(all_errors_combined)
x_min = 0

# Calculate histogram bins for all data to determine y limits
hist_counts = []
for errors in all_errors:
    counts, _ = np.histogram(errors, bins=50, range=(x_min, x_max))
    hist_counts.append(counts.max())
y_max = max(hist_counts) * 1.1  # Add 10% padding

for idx, (ax, errors, metrics, color, name) in enumerate(zip(axes, all_errors, all_metrics, colors, strategy_names)):
    # Histogram of errors
    ax.hist(errors, bins=50, range=(x_min, 1000), edgecolor='black', alpha=0.7, color=color)
    ax.axvline(metrics['mean'], color='lightcoral', linestyle='--', linewidth=2, 
               label=f'Mean: {metrics["mean"]:.1f} km')
    ax.axvline(metrics['median'], color='mediumturquoise', linestyle='--', linewidth=2, 
               label=f'Median: {metrics["median"]:.1f} km')
    
    # Set common axis limits
    ax.set_xlim(x_min, 1000)
    ax.set_ylim(0, y_max)
    
    ax.set_xlabel('Error Distance (km)', fontsize=11)
    ax.set_ylabel('Number of Documents', fontsize=11)
    ax.set_title(f'Strategy {idx+1}: {name}', fontsize=13, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Add text box with key metrics
    textstr = f'Acc@161km: {metrics["acc_161km"]:.1f}%'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.97, 0.97, textstr, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', horizontalalignment='right', bbox=props)

plt.tight_layout()
plt.show()


## Connecting to Spatial Search

In the spatial search notebook, we assumed all documents came with precise coordinates. This demo showed:

- For datasets without coordinates, geoparsing can automatically generate them from text
- Geoparsing gives us automation but we potentially lose some spatial precision
- Single documents often mention multiple places - choosing the "right" one is non-trivial
- Different applications may benefit from different resolution strategies